In [1]:
import os
import dotenv

from utils import get_algod_client
from account import Account

In [2]:
dotenv.load_dotenv('.env')

algod = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

creator = Account(os.environ.get('CREATOR_PK'))
pooler = Account(os.environ.get("SELLER_PK"))
swapper = Account(os.environ.get("BUYER_PK"))

app_id = int(os.environ.get('VALIDATOR_APP_ID'))
asset1_id = int(os.environ.get("ASSET1_ID"))

print(f"Creator's address: {creator.get_address()}")
print(f"Seller's address: {pooler.get_address()}")
print(f"Buyer's address: {swapper.get_address()}")
print(f"Validator App ID: {app_id}")
print(f"Asset 1 ID: {asset1_id}")

Creator's address: FE6UTVPOXD7HCTEYTG27P7KEDZCQFX7ECJANPTI76CMJSKKDQRYFVB4NWM
Seller's address: GYZIWMWOUMBIUUI3QV3VRWQYVJ5QGMO7CUW4RNMCIZKGJULFOFQFF3MCGA
Buyer's address: PAGKBMWTKVNLVPOT6JQLMMCEULFMOMGNU3JU6GFISDKTFMRAYOJPBARU5Y
Validator App ID: 6
Asset 1 ID: 4


In [3]:
from swap.operations.client import AlgoverseClient

client = AlgoverseClient(algod, app_id, pooler.get_address())

Check if the account is opted into Tinyman and optin if necessary

In [4]:
if not client.is_opted_in():
    print('Account not opted into app, opting in now..')
    transaction_group = client.prepare_app_optin_transactions()
    transaction_group.sign_with_private_key(pooler.get_address(), pooler.get_private_key())
    result = client.submit(transaction_group, wait=True)

Fetch our two assets of interest

In [5]:
AVUSDC = client.fetch_asset(asset1_id)
ALGO = client.fetch_asset(0)

Fetch the pool we will work with

In [6]:
pool = client.fetch_pool(AVUSDC, ALGO)

In [7]:
# transaction_group = pool.prepare_bootstrap_transactions()
# transaction_group.sign_with_private_key(pooler.get_address(), pooler.get_private_key())
# result = client.submit(transaction_group, wait=True)

Get a quote for supplying 1000.0 TinyUSDC

In [8]:
quote = pool.fetch_mint_quote(AVUSDC(1_000_000), ALGO(1_500_000), slippage=0.01)

print(quote)

MintQuote(amounts_in={Asset(AVUSDC - 4): AVUSDC('1'), Asset(ALGO - 0): ALGO('1.5')}, liquidity_asset_amount=AVPOOL('1.147432671705804998055100441'), slippage=0.01)


Check if we are happy with the quote..

In [9]:
if quote.amounts_in[ALGO] < 5_000_000:
    # Prepare the mint transactions from the quote and sign them
    transaction_group = pool.prepare_mint_transactions_from_quote(quote)
    transaction_group.sign_with_private_key(pooler.get_address(), pooler.get_private_key())
    result = client.submit(transaction_group, wait=True)

    # Check if any excess liquidity asset remaining after the mint
    excess = pool.fetch_excess_amounts()
    if pool.liquidity_asset in excess:
        amount = excess[pool.liquidity_asset]
        print(f'Excess: {amount}')
        if amount > 1_000_000:
            transaction_group = pool.prepare_redeem_transactions(amount)
            transaction_group.sign_with_private_key(pooler.get_address(), pooler.get_private_key())
            result = client.submit(transaction_group, wait=True)

Waiting for confirmation
Transaction 2TDM5XSXTYKOAKY7FXHJEONBX74IYJK4PG5VL4O37AIQTKS47ZDA confirmed in round 12808.
Excess: AVPOOL('0.121623')


In [10]:
info = pool.fetch_pool_position()
share = info['share'] * 100
print(f'Pool Tokens: {info[pool.liquidity_asset]}')
print(f'Assets: {info[AVUSDC]}, {info[ALGO]}')
print(f'Share of pool: {share:.3f}%')

Pool Tokens: AVPOOL('13.264766')
Assets: AVUSDC('10.28159982938664406538009644'), ALGO('17.34058170814945548772811890')
Share of pool: 99.083%


Get a quote for a swap of 1 ALGO to TINYUSDC with 1% slippage tolerance

In [11]:
quote = pool.fetch_fixed_input_swap_quote(ALGO(1_000_000), slippage=0.01)
print(quote)
print(f'AVUSDC per ALGO: {quote.price}')
print(f'AVUSDC per ALGO (worst case): {quote.price_with_slippage}')

SwapQuote(swap_type='fixed-input', amount_in=ALGO('1'), amount_out=AVUSDC('0.559281'), swap_fees=ALGO('0.003'), slippage=0.01)
AVUSDC per ALGO: 0.559281
AVUSDC per ALGO (worst case): 0.553689


In [12]:
client = AlgoverseClient(algod, app_id, swapper.get_address())

Check if the account is opted into Tinyman and optin if necessary

In [13]:
if(not client.is_opted_in()):
    print('Account not opted into app, opting in now..')
    transaction_group = client.prepare_app_optin_transactions()
    transaction_group.sign_with_private_key(swapper.get_address(), swapper.get_private_key())
    result = client.submit(transaction_group, wait=True)

Fetch the pool we will work with

In [14]:
pool = client.fetch_pool(AVUSDC, ALGO)

In [15]:
print(f'Swapping {quote.amount_in} to {quote.amount_out_with_slippage}')
if not client.asset_is_opted_in(AVUSDC.id, swapper.get_address()):
    transaction_group = client.prepare_asset_optin_transactions(AVUSDC.id)
    transaction_group.sign_with_private_key(swapper.get_address(), swapper.get_private_key())
    result = client.submit(transaction_group, wait=True)
# Prepare a transaction group
transaction_group = pool.prepare_swap_transactions_from_quote(quote)
# Sign the group with our key
transaction_group.sign_with_private_key(swapper.get_address(), swapper.get_private_key())
# Submit transactions to the network and wait for confirmation
result = client.submit(transaction_group, wait=True)

# Check if any excess remaining after the swap
excess = pool.fetch_excess_amounts()
if AVUSDC in excess:
    amount = excess[AVUSDC]
    print(f'Excess: {amount}')
    # We might just let the excess accumulate rather than redeeming if its < 1 TinyUSDC
    if amount > 1_000_000:
        transaction_group = pool.prepare_redeem_transactions(amount)
        transaction_group.sign_with_private_key(swapper.get_address(), swapper.get_private_key())
        result = client.submit(transaction_group, wait=True)

Swapping ALGO('1') to AVUSDC('0.553689')
Waiting for confirmation
Transaction VVNAFJMPFKAEPHZQA5V54CZVTL7JQVZRRTNSROVQRJGDV6H55OLA confirmed in round 12810.
Excess: AVUSDC('0.01182')
